In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')

# Pick date
date = feed.get_first_week()[0]
print('date', date)


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337
date 20140526


In [60]:
import copy


def proto1(feed, trip_stats, dates, split_directions=False,
  headway_start_time='07:00:00', headway_end_time='19:00:00'):
    frames = []
    for date in dates:
        f = feed.compute_route_stats(trip_stats, date,
          split_directions=split_directions,
          headway_start_time=headway_start_time,
          headway_end_time=headway_end_time)
        f['date'] = date
        frames.append(f)
    
    return pd.concat(frames)

def proto2(feed, dates, split_directions=False, freq='5Min'):
    """
    """
    cols = [
      'num_trips',
      ]
    if not dates:
        return pd.DataFrame([], columns=cols)

    if split_directions:
        cols.append('direction_id')

    activity = feed.compute_trip_activity(dates)

    # Collect stats for each date, memoizing stats by trip ID sequence
    # to avoid unnecessary recomputations.
    # Store in dictionary of the form
    # trip ID sequence ->
    # [stats DataFarme, date list that stats apply]
    stats_and_dates_by_ids = {}
    for date in dates:
        ids = tuple(activity.loc[activity[date] > 0, 'trip_id'])
        if ids in stats_and_dates_by_ids:
            # Append date to date list
            stats_and_dates_by_ids[ids][1].append(date)
        else:
            # Compute stats
            t = feed.trips
            trips = t[t['trip_id'].isin(ids)].copy()
            stats = gt.compute_stop_time_series_base(feed.stop_times, trips,
              split_directions=split_directions, freq=freq, date_label=date)

            # Remember stats
            stats_and_dates_by_ids[ids] = [stats, [date]]

    # Assemble stats into DataFrame
    frames = []
    for stats, dates in stats_and_dates_by_ids.values():
        if stats.empty:
            continue
        for date in dates:
            f = stats.copy()
            # Replace date
            d = gt.datestr_to_date(date)
            f.index = f.index.map(lambda t: t.replace(
              year=d.year, month=d.month, day=d.day))
            frames.append(f)
    f = pd.concat(frames).sort_index()

    return f


In [9]:
ts = feed.compute_trip_stats()
dates = feed.get_dates()
len(dates)

217

In [84]:
feed.compute_route_stats(ts, dates[0])

,date,route_id,route_short_name,route_type,num_trips,is_bidirectional,is_loop,start_time,end_time,max_headway,min_headway,mean_headway,peak_num_trips,peak_start_time,peak_end_time,service_duration,service_distance,service_speed,mean_trip_distance,mean_trip_duration


In [67]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

CPU times: user 896 ms, sys: 0 ns, total: 896 ms
Wall time: 895 ms


indicator           num_trips                                            \
stop_id                750000 750001 750002 750003 750004 750005 750006   
2014-05-26 00:00:00        13     13     13     13     13     13     12   
2014-05-26 12:00:00        17     17     17     17     17     17     18   

indicator                                 ...                                \
stop_id             750007 750008 750009  ...   750437 750440 750448 750449   
2014-05-26 00:00:00     12     12     12  ...        6      5     10    110   
2014-05-26 12:00:00     18     18     18  ...       10     12     11    179   

indicator                                                      
stop_id             750450 750452 750453 750454 750455 750456  
2014-05-26 00:00:00     36     28     25     11      4     46  
2014-05-26 12:00:00     65     53     49     17     11     79  

[2 rows x 416 columns]

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()